# Multi-Strategy Benchmark (Extended)

Compare **15 alpha extraction strategies** under identical walk-forward conditions.

| ID | Strategy | Architecture |
|----|----------|-------------|
| A | LSTM Baseline | Price-only temporal LSTM |
| B | NLP Only | Sentiment MLP |
| C | Late Ensemble | A + B with ridge meta |
| D | Residual Sentiment | Market-residualized NLP |
| E | Gated Hybrid | Gated price-NLP fusion |
| E1 | Attention Hybrid | Cross-attention price/NLP |
| E2 | Additive Residual | T + alpha*S fusion |
| F | CS-Attention LSTM | Temporal + stock attention |
| F1 | Temporal Fusion | Simplified TFT |
| G | Transformer | Encoder-only transformer |
| G1 | Efficient Transformer | Linear attention O(n*d) |
| H | Random Forest | sklearn RF (no torch) |
| I | LightGBM | Gradient boosted trees (no torch) |
| J | Short Horizon NLP | NLP MLP 5D only |
| K | Short Horizon Hybrid | Gated hybrid 5D only |

**Evaluation:** IC, Sharpe, R2, per-stock R2, directional accuracy, calibration, market-level prediction, backtest PnL, DM statistical tests.

**Goal:** Determine which structure produces real, production-surviving alpha.

In [ ]:
!pip install -q yfinance lightgbm torch optuna pyarrow scikit-learn scipy pandas numpy matplotlib

In [ ]:
import os
os.chdir("/content")
!rm -rf AI-stock-investment-tool

REPO = "https://github.com/kevin6598/AI-stock-investment-tool.git"
ret = os.system("git clone %s 2>/dev/null" % REPO)
if ret != 0:
    from getpass import getpass
    token = getpass("GitHub token (repo scope): ")
    os.system("git clone https://%s@github.com/kevin6598/AI-stock-investment-tool.git" % token)
    del token

os.chdir("/content/AI-stock-investment-tool")
!git log --oneline -3

In [ ]:
import torch, sys
print("Python: %s" % sys.version)
print("PyTorch: %s" % torch.__version__)
print("CUDA: %s" % torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU: %s" % torch.cuda.get_device_name(0))

## 1. Global Configuration

Single config block. All strategies respect these constraints.
No model-specific unfair tuning.

In [ ]:
CONFIG = {
    "horizons": [5, 21],
    "walk_forward": {
        "train_years": 2,
        "test_months": 4,
        "step_months": 4,
        "val_months": 2,
        "embargo_days": 5,
    },
    "max_epochs": 15,
    "early_stop_patience": 3,
    "ranking_weight": 0.5,
    "max_params": 1_500_000,
}

print("Horizons: %s" % CONFIG["horizons"])
print("Walk-forward: %s" % CONFIG["walk_forward"])
print("Max epochs: %d" % CONFIG["max_epochs"])
print("Max params: %d" % CONFIG["max_params"])

## 2. Standardized Data Pipeline

All strategies use the SAME processed dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_DIR = "/content/drive/MyDrive/ai_stock_tool"
DATA_PATH = os.path.join(DRIVE_DIR, "dataset.parquet")
ARTIFACT_DIR = os.path.join(DRIVE_DIR, "artifacts")
os.makedirs(ARTIFACT_DIR, exist_ok=True)
print("Data: %s" % DATA_PATH)

In [ ]:
import pandas as pd
import numpy as np

panel = pd.read_parquet(DATA_PATH)
valid_tickers = panel.index.get_level_values(1).unique().tolist()
print("Panel: %s" % str(panel.shape))
print("Tickers: %d" % len(valid_tickers))

date_min = panel.index.get_level_values(0).min()
date_max = panel.index.get_level_values(0).max()
data_span_months = (date_max - date_min).days / 30.44
print("Date range: %s to %s (%.0f months)" % (
    date_min.date(), date_max.date(), data_span_months))

# Estimate fold count
wf = CONFIG["walk_forward"]
train_months = wf["train_years"] * 12
val_months = wf.get("val_months", 3)
test_months = wf["test_months"]
step_months = wf["step_months"]
min_needed = train_months + val_months + test_months + 1
available_after_first = data_span_months - min_needed
est_folds = max(0, 1 + int(available_after_first / step_months))
print("\nFold estimate: train=%dmo + val=%dmo + test=%dmo = %dmo min" % (
    train_months, val_months, test_months, min_needed))
print("Expected folds: ~%d (with step=%dmo)" % (est_folds, step_months))
if est_folds == 0:
    print("WARNING: No folds possible! Reduce train_years or test_months.")

# Feature columns (same for ALL strategies)
feature_cols = [
    c for c in panel.columns
    if not c.startswith("fwd_return_")
    and not c.startswith("residual_return_")
    and not c.startswith("ranked_target_")
    and c not in ("_close", "ticker_id")
]

price_cols = [c for c in feature_cols if not c.startswith("nlp_")]
nlp_cols = [c for c in feature_cols if c.startswith("nlp_")]
print("\nFeatures: %d total (%d price, %d NLP)" % (
    len(feature_cols), len(price_cols), len(nlp_cols)))

# Check required targets
for h in CONFIG["horizons"]:
    tc = "fwd_return_%dd" % h
    if tc in panel.columns:
        non_null = panel[tc].notna().sum()
        print("  %s: %d non-null" % (tc, non_null))
    else:
        print("  WARNING: %s not found!" % tc)

In [ ]:
# Precompute derived features for all strategies

# Sentiment residual (for Strategy D)
market_feat = None
for cand in ["market_return", "market_return_21d", "spy_return_21d"]:
    if cand in panel.columns:
        market_feat = cand
        break

if market_feat:
    print("Market return feature: %s" % market_feat)
else:
    print("WARNING: No market return feature found for residualization")

# Volatility proxy
vol_feat = None
for cand in ["volatility_21d", "realized_vol_21d", "atr_pct"]:
    if cand in panel.columns:
        vol_feat = cand
        break

if vol_feat:
    print("Volatility feature: %s" % vol_feat)
else:
    print("WARNING: No volatility feature found")

# Data quality check
nan_pct = panel[feature_cols].isna().mean().mean() * 100
print("\nNaN rate: %.2f%%" % nan_pct)
print("Data pipeline ready.")

## 3. Strategy Definitions

All 15 strategies with identical interface: `train()`, `predict()`, `num_parameters()`.

- **A-F**: Neural temporal/NLP models (require PyTorch)
- **E1, E2**: Novel fusion architectures (cross-attention, additive residual)
- **F1**: Simplified Temporal Fusion Transformer
- **G, G1**: Transformer variants (standard + linear attention)
- **H, I**: Tree-based models (no torch required)
- **J, K**: Short-horizon specialists (5D only)

In [ ]:
from training.strategy_benchmark import (
    BenchmarkConfig, BenchmarkEvaluator, STRATEGY_REGISTRY,
    save_benchmark_results, run_integrity_checks,
    ExtendedMetrics, DMTestResult,
)

config = BenchmarkConfig.from_dict(CONFIG)

# List all 15 strategies
print("Strategies to benchmark (%d total):" % len(STRATEGY_REGISTRY))
print("-" * 50)
for key, cls in sorted(STRATEGY_REGISTRY.items()):
    sh = getattr(cls, 'supported_horizons', None)
    h_info = "all horizons" if sh is None else "%s only" % sh
    print("  %-3s: %-28s (%s)" % (key, cls.name, h_info))

# Verify param counts (dry-run instantiation)
print("\nParam count check (instantiation only):")
for key, cls in sorted(STRATEGY_REGISTRY.items()):
    s = cls()
    print("  %s: OK" % cls.name)

print("\nTotal: %d strategies" % len(STRATEGY_REGISTRY))

## 4. Run Benchmark

Walk-forward evaluation for every strategy at every horizon.

Expected runtime: ~45-60 min on T4 GPU (15 strategies x 2 horizons).

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(message)s", datefmt="%H:%M:%S")

evaluator = BenchmarkEvaluator(panel, feature_cols, config)

strategy_classes = list(STRATEGY_REGISTRY.values())
print("Running %d strategies x %d horizons..." % (
    len(strategy_classes), len(config.horizons)))
print("=" * 60)

import time
t0 = time.time()
results = evaluator.run_all(strategy_classes)
total_time = time.time() - t0

print("\n" + "=" * 60)
print("Benchmark complete in %.1f min" % (total_time / 60))
print("Total evaluations: %d" % len(results))
print("Strategies with extended metrics: %d" % sum(1 for r in results if r.extended))

## 5. Strategy Comparison Table

In [ ]:
import math

# Build comparison table
rows = []
for r in results:
    rows.append({
        "Strategy": r.name,
        "Horizon": r.horizon,
        "IC": round(r.ic_mean, 4),
        "ICIR": round(r.icir, 2),
        "Sharpe": round(r.sharpe, 2),
        "IC_std": round(r.ic_std, 4),
        "Prod IC": round(r.prod_ic, 4),
        "Overfit": round(r.overfit_score, 3),
        "Composite": round(r.composite, 4),
        "Params": r.param_count,
        "Time(s)": round(r.train_time, 1),
        "Status": r.status,
    })

df_results = pd.DataFrame(rows)
print("\nStrategy Benchmark Results (sorted by Composite):")
print("=" * 100)
display(df_results)

# Summary by status
print("\nStatus Summary:")
for status in ["PASS", "WARN", "FAIL"]:
    count = sum(1 for r in results if r.status == status)
    names = [r.name + "/" + r.horizon for r in results if r.status == status]
    print("  %s (%d): %s" % (status, count, ", ".join(names) if names else "none"))

## 5a. Extended Metrics Table

Regression accuracy, directional metrics, calibration, and market-level predictions.

In [ ]:
# Extended Metrics: Regression, Directional, Calibration, Market-level
ext_rows = []
for r in results:
    if r.extended is None:
        continue
    e = r.extended
    ext_rows.append({
        "Strategy": r.name,
        "Horizon": r.horizon,
        "RMSE": round(e.rmse, 6),
        "R2": round(e.r_squared, 4),
        "Hit%": round(e.hit_ratio * 100, 1),
        "Prec": round(e.precision, 3),
        "Recall": round(e.recall, 3),
        "F1": round(e.f1, 3),
        "Cal.Slope": round(e.calib_slope, 3),
        "Mkt.R2": round(e.market_r2, 4),
        "Mkt.Dir%": round(e.market_direction_accuracy * 100, 1),
    })

if ext_rows:
    df_ext = pd.DataFrame(ext_rows)
    print("Extended Metrics:")
    print("=" * 120)
    display(df_ext)
else:
    print("No extended metrics available.")

In [ ]:
# Per-Stock R2 Summary
r2_rows = []
for r in results:
    if r.extended is None:
        continue
    e = r.extended
    r2_rows.append({
        "Strategy": r.name,
        "Horizon": r.horizon,
        "Mean R2": round(e.mean_stock_r2, 4),
        "Median R2": round(e.median_stock_r2, 4),
        "% Positive": round(e.pct_r2_positive, 1),
        "% > 0.05": round(e.pct_r2_above_005, 1),
        "N Stocks": len(e.stock_r2_values),
    })

if r2_rows:
    df_r2 = pd.DataFrame(r2_rows)
    print("Per-Stock R2 Summary:")
    print("=" * 90)
    display(df_r2)
else:
    print("No per-stock R2 data available.")

In [ ]:
# Backtest Results: Long-Only and Long-Short
bt_rows = []
for r in results:
    if r.extended is None:
        continue
    e = r.extended
    bt_rows.append({
        "Strategy": r.name,
        "Horizon": r.horizon,
        "L/O CAGR": "%.2f%%" % (e.lo_cagr * 100),
        "L/O Sharpe": round(e.lo_sharpe, 2),
        "L/O MaxDD": "%.2f%%" % (e.lo_max_dd * 100),
        "L/S CAGR": "%.2f%%" % (e.ls_cagr * 100),
        "L/S Sharpe": round(e.ls_sharpe, 2),
        "L/S MaxDD": "%.2f%%" % (e.ls_max_dd * 100),
    })

if bt_rows:
    df_bt = pd.DataFrame(bt_rows)
    print("Backtest Results (Long-Only and Long-Short 20/20):")
    print("=" * 100)
    display(df_bt)
else:
    print("No backtest data available.")

In [ ]:
# Diebold-Mariano Test Matrix
dm_results = BenchmarkEvaluator.compute_dm_tests(results)

if dm_results:
    # Display as table
    dm_rows = []
    for dm in dm_results:
        dm_rows.append({
            "A": dm.strategy_a.split("_")[0],
            "B": dm.strategy_b.split("_")[0],
            "Horizon": dm.horizon,
            "DM Stat": round(dm.dm_stat, 2),
            "p-value": round(dm.p_value, 4),
            "Better": dm.better.split("_")[0],
            "Sig?": "*" if dm.p_value < 0.05 else ("." if dm.p_value < 0.10 else ""),
        })
    df_dm = pd.DataFrame(dm_rows)
    print("Diebold-Mariano Tests (* = p<0.05, . = p<0.10):")
    print("=" * 80)
    display(df_dm)

    # Heatmap per horizon
    import matplotlib.pyplot as plt
    horizons = sorted(set(dm.horizon for dm in dm_results))
    fig_dm, axes_dm = plt.subplots(1, len(horizons), figsize=(8 * len(horizons), 6))
    if len(horizons) == 1:
        axes_dm = [axes_dm]

    for ax, h in zip(axes_dm, horizons):
        h_dm = [dm for dm in dm_results if dm.horizon == h]
        names = sorted(set(
            [dm.strategy_a.split("_")[0] for dm in h_dm] +
            [dm.strategy_b.split("_")[0] for dm in h_dm]
        ))
        n = len(names)
        pval_matrix = np.ones((n, n))
        name_to_idx = {nm: i for i, nm in enumerate(names)}

        for dm in h_dm:
            ia = name_to_idx.get(dm.strategy_a.split("_")[0])
            ib = name_to_idx.get(dm.strategy_b.split("_")[0])
            if ia is not None and ib is not None:
                pval_matrix[ia, ib] = dm.p_value
                pval_matrix[ib, ia] = dm.p_value

        im = ax.imshow(pval_matrix, cmap="RdYlGn", vmin=0, vmax=0.2)
        ax.set_xticks(range(n))
        ax.set_xticklabels(names, rotation=45, fontsize=7, ha="right")
        ax.set_yticks(range(n))
        ax.set_yticklabels(names, fontsize=7)
        ax.set_title("DM p-values: %s" % h, fontweight="bold")

        for i in range(n):
            for j in range(n):
                if i != j:
                    ax.text(j, i, "%.2f" % pval_matrix[i, j],
                            ha="center", va="center", fontsize=6,
                            color="white" if pval_matrix[i, j] < 0.05 else "black")

    plt.colorbar(im, ax=axes_dm[-1], shrink=0.7, label="p-value")
    plt.tight_layout()
    plt.savefig(os.path.join(ARTIFACT_DIR, "dm_test_heatmap.png"), dpi=150, bbox_inches="tight")
    plt.show()
    print("DM test heatmap saved.")
else:
    print("No DM test results (need fold predictions from multiple strategies).")

## 6. Visualization Dashboard

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(20, 16))
gs_layout = gridspec.GridSpec(2, 3, hspace=0.35, wspace=0.3)

# --- Panel 1: IC Comparison Bar Chart ---
ax1 = fig.add_subplot(gs_layout[0, 0])
labels = ["%s\n%s" % (r.name.split("_", 1)[-1][:12], r.horizon) for r in results]
ics = [r.ic_mean for r in results]
colors = ["#4CAF50" if r.status == "PASS" else "#FFC107" if r.status == "WARN" else "#F44336"
          for r in results]
ax1.barh(range(len(results)), ics, color=colors, edgecolor="white")
ax1.set_yticks(range(len(results)))
ax1.set_yticklabels(labels, fontsize=7)
ax1.set_xlabel("Cross-Sectional IC")
ax1.set_title("IC Comparison", fontweight="bold")
ax1.axvline(x=0, color="gray", linewidth=0.5)
ax1.invert_yaxis()

# --- Panel 2: IC Stability (fold ICs per strategy) ---
ax2 = fig.add_subplot(gs_layout[0, 1])
cmap = plt.colormaps["tab20"]
for i, r in enumerate(results):
    if r.fold_metrics:
        fold_ics = [f.ic for f in r.fold_metrics]
        x_positions = [i] * len(fold_ics)
        ax2.scatter(x_positions, fold_ics, color=cmap(i % 20), s=30, zorder=3, alpha=0.7)
        ax2.plot([i, i], [min(fold_ics), max(fold_ics)],
                 color=cmap(i % 20), linewidth=2, alpha=0.5)
ax2.set_xticks(range(len(results)))
ax2.set_xticklabels([r.name.split("_")[0] + "/" + r.horizon for r in results],
                     rotation=90, fontsize=6, ha="center")
ax2.axhline(y=0, color="gray", linewidth=0.5, linestyle="--")
ax2.set_ylabel("IC per fold")
ax2.set_title("IC Stability (per fold)", fontweight="bold")
ax2.grid(axis="y", alpha=0.3)

# --- Panel 3: Composite Score Bar ---
ax3 = fig.add_subplot(gs_layout[0, 2])
composites = [r.composite if not math.isinf(r.composite) else 0 for r in results]
ax3.barh(range(len(results)), composites, color=colors, edgecolor="white")
ax3.set_yticks(range(len(results)))
ax3.set_yticklabels(labels, fontsize=7)
ax3.set_xlabel("Composite Score")
ax3.set_title("Composite Score", fontweight="bold")
ax3.axvline(x=0, color="gray", linewidth=0.5)
ax3.invert_yaxis()

# --- Panel 4: Gate Activation ---
ax4 = fig.add_subplot(gs_layout[1, 0])
gate_data = [(r.name, r.horizon, r.gate_stats) for r in results if r.gate_stats]
if gate_data:
    g_labels = ["%s/%s" % (n, h) for n, h, _ in gate_data]
    g_means = []
    g_stds = []
    for _, _, gstat in gate_data:
        if isinstance(gstat, dict):
            g_means.append(gstat.get("gate_mean", 0))
            g_stds.append(gstat.get("gate_std", 0))
        else:
            g_means.append(0.5)
            g_stds.append(0)
    ax4.barh(g_labels, g_means, xerr=g_stds, color="#2196F3", capsize=5)
    ax4.set_xlabel("Gate Activation")
    ax4.set_title("Gate Distribution (Hybrid)", fontweight="bold")
    ax4.set_xlim(0, 1)
else:
    ax4.text(0.5, 0.5, "No gate stats\navailable",
             transform=ax4.transAxes, ha="center", va="center", fontsize=14, color="gray")
    ax4.set_title("Gate Distribution", fontweight="bold")

# --- Panel 5: Ensemble Weights ---
ax5 = fig.add_subplot(gs_layout[1, 1])
ens_data = [(r.name, r.horizon, r.ensemble_weights) for r in results if r.ensemble_weights]
if ens_data:
    for i, (name, horizon, w) in enumerate(ens_data):
        x = [0, 1]
        vals = [w.get("lstm", 0), w.get("nlp", 0)]
        ax5.bar([v + i * 0.3 for v in x], vals, width=0.25,
                label="%s/%s" % (name.split("_")[0], horizon))
    ax5.set_xticks([0, 1])
    ax5.set_xticklabels(["LSTM weight", "NLP weight"])
    ax5.set_title("Ensemble Weights", fontweight="bold")
    ax5.legend(fontsize=8)
    ax5.axhline(y=0, color="gray", linewidth=0.5, linestyle="--")
else:
    ax5.text(0.5, 0.5, "No ensemble weights\navailable",
             transform=ax5.transAxes, ha="center", va="center", fontsize=14, color="gray")
    ax5.set_title("Ensemble Weights", fontweight="bold")

# --- Panel 6: Horizon Sensitivity ---
ax6 = fig.add_subplot(gs_layout[1, 2])
strategy_names = sorted(set(r.name for r in results))
horizon_labels = sorted(set(r.horizon for r in results))
x_pos = np.arange(len(strategy_names))
width = 0.35

for j, h in enumerate(horizon_labels):
    h_ics = []
    for sn in strategy_names:
        match = [r for r in results if r.name == sn and r.horizon == h]
        h_ics.append(match[0].ic_mean if match else 0)
    offset = (j - len(horizon_labels) / 2 + 0.5) * width
    ax6.bar(x_pos + offset, h_ics, width, label=h)

ax6.set_xticks(x_pos)
ax6.set_xticklabels([n.split("_")[0] for n in strategy_names],
                     rotation=45, fontsize=7, ha="right")
ax6.set_ylabel("IC")
ax6.set_title("Horizon Sensitivity (5D vs 21D)", fontweight="bold")
ax6.legend(fontsize=9)
ax6.axhline(y=0, color="gray", linewidth=0.5, linestyle="--")
ax6.grid(axis="y", alpha=0.3)

fig.suptitle("Multi-Strategy Benchmark Dashboard (15 Models)", fontsize=16, fontweight="bold", y=1.01)
plt.savefig(os.path.join(ARTIFACT_DIR, "strategy_benchmark.png"), dpi=150, bbox_inches="tight")
plt.show()
print("Dashboard saved.")

## 6a. Extended Visualizations

Five additional panels: Per-Stock R2 Histogram, Pred vs Actual Scatter, Calibration Slope, Rolling R2, Cumulative PnL.

In [ ]:
# --- Visualization 1: Per-Stock R2 Histogram ---
ext_results = [r for r in results if r.extended and r.extended.stock_r2_values]
n_ext = len(ext_results)

if n_ext > 0:
    ncols = min(5, n_ext)
    nrows = (n_ext + ncols - 1) // ncols
    fig_r2h, axes_r2h = plt.subplots(nrows, ncols, figsize=(4 * ncols, 3 * nrows))
    if n_ext == 1:
        axes_flat = [axes_r2h]
    else:
        axes_flat = axes_r2h.flatten() if hasattr(axes_r2h, 'flatten') else [axes_r2h]

    for idx, r in enumerate(ext_results):
        if idx >= len(axes_flat):
            break
        ax = axes_flat[idx]
        vals = list(r.extended.stock_r2_values.values())
        ax.hist(vals, bins=20, color="#2196F3", edgecolor="white", alpha=0.8)
        ax.axvline(x=0, color="red", linewidth=1, linestyle="--")
        ax.axvline(x=np.median(vals), color="green", linewidth=1, linestyle="-",
                   label="median=%.3f" % np.median(vals))
        ax.set_title("%s/%s" % (r.name.split("_")[0], r.horizon), fontsize=9)
        ax.set_xlabel("R2", fontsize=8)
        ax.legend(fontsize=6)

    for idx in range(n_ext, len(axes_flat)):
        axes_flat[idx].set_visible(False)

    fig_r2h.suptitle("Per-Stock R2 Distribution", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.savefig(os.path.join(ARTIFACT_DIR, "per_stock_r2_histogram.png"),
                dpi=150, bbox_inches="tight")
    plt.show()
    print("Per-stock R2 histogram saved.")
else:
    print("No per-stock R2 data for visualization.")

In [ ]:
# --- Visualization 2: Pred vs Actual Scatter ---
scatter_results = [r for r in results if r.fold_predictions]

if scatter_results:
    n_sc = len(scatter_results)
    ncols = min(5, n_sc)
    nrows = (n_sc + ncols - 1) // ncols
    fig_sc, axes_sc = plt.subplots(nrows, ncols, figsize=(4 * ncols, 4 * nrows))
    if n_sc == 1:
        axes_flat = [axes_sc]
    else:
        axes_flat = axes_sc.flatten() if hasattr(axes_sc, 'flatten') else [axes_sc]

    for idx, r in enumerate(scatter_results):
        if idx >= len(axes_flat):
            break
        ax = axes_flat[idx]
        all_p = np.concatenate([fp.predictions for fp in r.fold_predictions])
        all_a = np.concatenate([fp.actuals for fp in r.fold_predictions])
        valid = ~(np.isnan(all_p) | np.isnan(all_a))
        p, a = all_p[valid], all_a[valid]

        if len(p) > 5000:
            idx_sub = np.random.choice(len(p), 5000, replace=False)
            p_plot, a_plot = p[idx_sub], a[idx_sub]
        else:
            p_plot, a_plot = p, a

        ax.scatter(p_plot, a_plot, alpha=0.1, s=3, color="#2196F3")

        if len(p) > 10:
            coef = np.polyfit(p, a, 1)
            x_line = np.linspace(p.min(), p.max(), 50)
            ax.plot(x_line, coef[0] * x_line + coef[1], "r-", linewidth=1.5,
                    label="slope=%.2f" % coef[0])

        ax.set_title("%s/%s" % (r.name.split("_")[0], r.horizon), fontsize=9)
        ax.set_xlabel("Predicted", fontsize=8)
        ax.set_ylabel("Actual", fontsize=8)
        ax.legend(fontsize=7)

    for idx in range(n_sc, len(axes_flat)):
        axes_flat[idx].set_visible(False)

    fig_sc.suptitle("Prediction vs Actual", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.savefig(os.path.join(ARTIFACT_DIR, "pred_vs_actual_scatter.png"),
                dpi=150, bbox_inches="tight")
    plt.show()
    print("Pred vs actual scatter saved.")
else:
    print("No fold predictions for scatter plot.")

In [ ]:
# --- Visualization 3: Calibration Slope Bar Chart ---
cal_results = [r for r in results if r.extended is not None]

if cal_results:
    fig_cal, ax_cal = plt.subplots(figsize=(10, max(4, len(cal_results) * 0.4)))
    labels_cal = ["%s/%s" % (r.name.split("_")[0], r.horizon) for r in cal_results]
    slopes = [r.extended.calib_slope for r in cal_results]
    colors_cal = ["#4CAF50" if abs(s - 1.0) < 0.3 else "#FFC107" if abs(s - 1.0) < 0.6
                  else "#F44336" for s in slopes]

    y_pos = range(len(cal_results))
    ax_cal.barh(y_pos, slopes, color=colors_cal, edgecolor="white")
    ax_cal.set_yticks(y_pos)
    ax_cal.set_yticklabels(labels_cal, fontsize=8)
    ax_cal.axvline(x=1.0, color="blue", linewidth=2, linestyle="--", label="Ideal (slope=1.0)")
    ax_cal.set_xlabel("Calibration Slope")
    ax_cal.set_title("Calibration: Pred vs Actual Regression Slope", fontweight="bold")
    ax_cal.legend(fontsize=9)
    ax_cal.invert_yaxis()
    plt.tight_layout()
    plt.savefig(os.path.join(ARTIFACT_DIR, "calibration_slope.png"), dpi=150, bbox_inches="tight")
    plt.show()
    print("Calibration slope chart saved.")
else:
    print("No calibration data for visualization.")

In [ ]:
# --- Visualization 4: Rolling R2 (126-sample window) ---
rolling_results = [r for r in results if r.fold_predictions]

if rolling_results:
    fig_roll, ax_roll = plt.subplots(figsize=(14, 6))
    window = 126

    for r in rolling_results:
        all_p = np.concatenate([fp.predictions for fp in r.fold_predictions])
        all_a = np.concatenate([fp.actuals for fp in r.fold_predictions])
        valid = ~(np.isnan(all_p) | np.isnan(all_a))
        p, a = all_p[valid], all_a[valid]

        if len(p) < window + 10:
            continue

        rolling_r2 = []
        for i in range(window, len(p)):
            p_w = p[i - window:i]
            a_w = a[i - window:i]
            ss_res = np.sum((p_w - a_w) ** 2)
            ss_tot = np.sum((a_w - np.mean(a_w)) ** 2)
            r2 = 1.0 - ss_res / ss_tot if ss_tot > 1e-10 else 0.0
            rolling_r2.append(r2)

        label = "%s/%s" % (r.name.split("_")[0], r.horizon)
        ax_roll.plot(rolling_r2, label=label, alpha=0.7, linewidth=1)

    ax_roll.axhline(y=0, color="gray", linewidth=0.5, linestyle="--")
    ax_roll.set_xlabel("Sample Index")
    ax_roll.set_ylabel("Rolling R2 (window=%d)" % window)
    ax_roll.set_title("Rolling R2 Across Walk-Forward Test Periods", fontweight="bold")
    ax_roll.legend(fontsize=7, ncol=3, loc="upper right")
    ax_roll.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(ARTIFACT_DIR, "rolling_r2.png"), dpi=150, bbox_inches="tight")
    plt.show()
    print("Rolling R2 chart saved.")
else:
    print("No fold predictions for rolling R2.")

In [ ]:
# --- Visualization 5: Cumulative PnL Curves ---
pnl_results = [r for r in results
               if r.extended is not None
               and r.extended.lo_equity is not None
               and len(r.extended.lo_equity) > 1]

if pnl_results:
    fig_pnl, (ax_lo, ax_ls) = plt.subplots(1, 2, figsize=(16, 6))

    for r in pnl_results:
        label = "%s/%s" % (r.name.split("_")[0], r.horizon)
        ax_lo.plot(r.extended.lo_equity, label=label, alpha=0.7, linewidth=1)
        if r.extended.ls_equity is not None and len(r.extended.ls_equity) > 1:
            ax_ls.plot(r.extended.ls_equity, label=label, alpha=0.7, linewidth=1)

    ax_lo.axhline(y=1.0, color="gray", linewidth=0.5, linestyle="--")
    ax_lo.set_xlabel("Trading Day")
    ax_lo.set_ylabel("Equity")
    ax_lo.set_title("Long-Only Cumulative PnL", fontweight="bold")
    ax_lo.legend(fontsize=6, ncol=2)
    ax_lo.grid(alpha=0.3)

    ax_ls.axhline(y=1.0, color="gray", linewidth=0.5, linestyle="--")
    ax_ls.set_xlabel("Trading Day")
    ax_ls.set_ylabel("Equity")
    ax_ls.set_title("Long-Short (20/20) Cumulative PnL", fontweight="bold")
    ax_ls.legend(fontsize=6, ncol=2)
    ax_ls.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(ARTIFACT_DIR, "cumulative_pnl.png"), dpi=150, bbox_inches="tight")
    plt.show()
    print("Cumulative PnL curves saved.")
else:
    print("No equity curve data for PnL visualization.")

## 7. Diagnostic Output

In [ ]:
# Save full benchmark results to JSON
save_path = save_benchmark_results(
    results,
    path=os.path.join(ARTIFACT_DIR, "strategy_benchmark_results.json"),
)
print("Results saved to: %s" % save_path)

# Print per-strategy detail
print("\n" + "=" * 70)
print("DETAILED RESULTS")
print("=" * 70)
for r in results:
    print("\n--- %s / %s [%s] ---" % (r.name, r.horizon, r.status))
    print("  IC: %.4f +/- %.4f  ICIR: %.2f" % (r.ic_mean, r.ic_std, r.icir))
    print("  Sharpe: %.2f  MaxDD: %.4f" % (r.sharpe, r.max_drawdown))
    print("  Overfit: %.3f  Composite: %.4f" % (r.overfit_score, r.composite))
    print("  Prod IC: %.4f  Params: %d  Time: %.1fs" % (
        r.prod_ic, r.param_count, r.train_time))
    if r.fold_metrics:
        fold_ics = [f.ic for f in r.fold_metrics]
        print("  Fold ICs: %s" % [round(x, 4) for x in fold_ics])
    if r.extended:
        e = r.extended
        print("  R2: %.4f  Hit%%: %.1f%%  Cal.Slope: %.3f" % (
            e.r_squared, e.hit_ratio * 100, e.calib_slope))
        print("  L/O Sharpe: %.2f  L/S Sharpe: %.2f" % (e.lo_sharpe, e.ls_sharpe))
    if r.gate_stats:
        print("  Gate: mean=%.3f std=%.3f" % (
            r.gate_stats.get("gate_mean", 0), r.gate_stats.get("gate_std", 0)))
    if r.ensemble_weights:
        w = r.ensemble_weights
        print("  Ensemble: lstm=%.3f nlp=%.3f intercept=%.4f" % (
            w.get("lstm", 0), w.get("nlp", 0), w.get("intercept", 0)))

## 8. Experimental Integrity Checks

In [ ]:
warnings = run_integrity_checks(results)

if warnings:
    print("INTEGRITY WARNINGS (%d):" % len(warnings))
    for w in warnings:
        print("  [!] %s" % w)
else:
    print("All integrity checks passed.")

# Additional checks
print("\nConsistency Checks:")

for h in sorted(set(r.horizon for r in results)):
    fold_counts = [len(r.fold_metrics) for r in results if r.horizon == h]
    if len(set(fold_counts)) > 1:
        print("  [!] Inconsistent fold counts at %s: %s" % (h, fold_counts))
    else:
        print("  [OK] %s: %d folds for all strategies" % (h, fold_counts[0] if fold_counts else 0))

for r in results:
    if r.param_count > CONFIG["max_params"]:
        print("  [!] %s: %d params > %d limit" % (r.name, r.param_count, CONFIG["max_params"]))

print("\nIntegrity check complete.")

## Summary & Decision

This benchmark answers:

1. **Does sentiment add incremental alpha?** Compare A vs C/E
2. **Is hybrid destructive or additive?** Compare A vs E/E1/E2
3. **Is ensemble safer than fusion?** Compare C vs E
4. **Cross-attention vs additive fusion?** Compare E1 vs E2
5. **Transformer vs LSTM?** Compare G vs A
6. **Trees vs neural networks?** Compare H/I vs A
7. **Which structure survives production retrain?** Check Prod IC column
8. **Statistical significance?** DM test p-values

In [ ]:
# Extended decision summary
print("=" * 60)
print("PRICE PREDICTION SUMMARY")
print("=" * 60)

# Best by extended metrics
ext_results_all = [r for r in results if r.extended is not None]
if ext_results_all:
    best_r2 = max(ext_results_all, key=lambda r: r.extended.mean_stock_r2)
    print("\nBest by Mean Stock R2: %s/%s (%.4f)" % (
        best_r2.name, best_r2.horizon, best_r2.extended.mean_stock_r2))

    best_hit = max(ext_results_all, key=lambda r: r.extended.hit_ratio)
    print("Best by Directional Accuracy: %s/%s (%.1f%%)" % (
        best_hit.name, best_hit.horizon, best_hit.extended.hit_ratio * 100))

    best_cal = min(ext_results_all, key=lambda r: abs(r.extended.calib_slope - 1.0))
    print("Best Calibration (slope closest to 1): %s/%s (%.3f)" % (
        best_cal.name, best_cal.horizon, best_cal.extended.calib_slope))

    best_lo = max(ext_results_all, key=lambda r: r.extended.lo_sharpe)
    print("Highest Sharpe (L/O): %s/%s (%.2f)" % (
        best_lo.name, best_lo.horizon, best_lo.extended.lo_sharpe))

    best_ls = max(ext_results_all, key=lambda r: r.extended.ls_sharpe)
    print("Highest Sharpe (L/S): %s/%s (%.2f)" % (
        best_ls.name, best_ls.horizon, best_ls.extended.ls_sharpe))

# DM significance vs baseline
if dm_results:
    baseline_name = "A_LSTM_Baseline"
    sig_improvements = [dm for dm in dm_results
                        if dm.p_value < 0.05
                        and (dm.strategy_a == baseline_name or dm.strategy_b == baseline_name)
                        and dm.better != baseline_name]
    if sig_improvements:
        print("\nDM test: significant improvements over baseline:")
        for dm in sig_improvements:
            print("  %s beats %s at %s (p=%.4f)" % (
                dm.better, baseline_name, dm.horizon, dm.p_value))
    else:
        print("\nDM test: no strategy significantly beats baseline (p<0.05)")

# Key comparisons
print("\n--- Key Comparisons ---")

# Sentiment alpha
a_21 = [r for r in results if r.name.startswith("A_") and r.horizon == "21D"]
c_21 = [r for r in results if r.name.startswith("C_") and r.horizon == "21D"]
e_21 = [r for r in results if r.name.startswith("E_") and r.horizon == "21D"]
if a_21 and c_21:
    delta = c_21[0].ic_mean - a_21[0].ic_mean
    verdict = "YES (+%.4f IC)" % delta if delta > 0.005 else "NO (delta=%.4f)" % delta
    print("1. Sentiment adds alpha? %s" % verdict)

# Hybrid vs baseline
if a_21 and e_21:
    delta = e_21[0].ic_mean - a_21[0].ic_mean
    verdict = "ADDITIVE (+%.4f)" % delta if delta > 0 else "DESTRUCTIVE (%.4f)" % delta
    print("2. Hybrid vs LSTM? %s" % verdict)

# Cross-attention vs additive
e1_21 = [r for r in results if r.name.startswith("E1") and r.horizon == "21D"]
e2_21 = [r for r in results if r.name.startswith("E2") and r.horizon == "21D"]
if e1_21 and e2_21:
    if e1_21[0].ic_mean > e2_21[0].ic_mean:
        print("3. Cross-attn vs Additive? CROSS-ATTN (IC %.4f vs %.4f)" % (
            e1_21[0].ic_mean, e2_21[0].ic_mean))
    else:
        print("3. Cross-attn vs Additive? ADDITIVE (IC %.4f vs %.4f)" % (
            e2_21[0].ic_mean, e1_21[0].ic_mean))

# Transformer vs LSTM
g_21 = [r for r in results if r.name.startswith("G_") and r.horizon == "21D"]
if a_21 and g_21:
    delta = g_21[0].ic_mean - a_21[0].ic_mean
    verdict = "TRANSFORMER (+%.4f)" % delta if delta > 0 else "LSTM (%.4f)" % delta
    print("4. Transformer vs LSTM? %s" % verdict)

# Trees vs neural
h_21 = [r for r in results if r.name.startswith("H_") and r.horizon == "21D"]
i_21 = [r for r in results if r.name.startswith("I_") and r.horizon == "21D"]
if a_21 and h_21 and i_21:
    best_tree = max([h_21[0], i_21[0]], key=lambda r: r.ic_mean)
    delta = best_tree.ic_mean - a_21[0].ic_mean
    verdict = "TREES (+%.4f)" % delta if delta > 0 else "NEURAL (%.4f)" % delta
    print("5. Trees vs Neural? %s (best tree: %s)" % (verdict, best_tree.name))

# Production survival
print("\n--- Production Survival ---")
for r in results:
    if r.ic_mean > 0 and r.prod_ic > 0:
        ratio = r.prod_ic / r.ic_mean if r.ic_mean > 1e-8 else 0
        survived = "SURVIVED" if ratio >= 0.9 else "DEGRADED"
        print("  %s/%s: WF IC=%.4f -> Prod IC=%.4f (%.0f%%) [%s]" % (
            r.name, r.horizon, r.ic_mean, r.prod_ic, ratio * 100, survived))

# Final recommendation
print("\n" + "=" * 60)
passing = [r for r in results if r.status == "PASS"]
if passing:
    best = passing[0]
    print("RECOMMENDATION: Deploy %s" % best.name)
    print("  Composite: %.4f  IC: %.4f  Prod IC: %.4f" % (
        best.composite, best.ic_mean, best.prod_ic))
    if best.extended:
        print("  R2: %.4f  Hit%%: %.1f%%  L/S Sharpe: %.2f" % (
            best.extended.r_squared, best.extended.hit_ratio * 100,
            best.extended.ls_sharpe))
else:
    warning_results = [r for r in results if r.status == "WARN"]
    if warning_results:
        print("RECOMMENDATION: Cautiously deploy %s (WARN status)" % warning_results[0].name)
    else:
        print("RECOMMENDATION: No viable strategy. Review data/features.")
print("=" * 60)